## **Noise Models**

##### To test the functionality of the **NoiseModels** in isolation from the rest of the API. This is to confirm that noise is applied correctly to observations and that the result is valid and clipped to the observation space bounds.

#### Gaussian Model

In [1]:
import importlib
import Failure_API.tests.utils.dummy_env as dv
import Failure_API.src.wrapper_api.models.noise_model as nm
import Failure_API.src.wrapper_api.failure_apis.noise_wrapper as nw
import Failure_API.src.wrapper_api.failure_apis.sharedobs_wrapper as swr
from Failure_API.src.wrapper_api.failure_apis.sharedobs_wrapper import SharedObsWrapper

importlib.reload(nm)
importlib.reload(nw)
importlib.reload(dv)
importlib.reload(swr)
from Failure_API.tests.utils.dummy_env import DummyEnv
from Failure_API.src.wrapper_api.failure_apis.noise_wrapper import NoiseWrapper
from Failure_API.src.wrapper_api.models.noise_model import GaussianNoise, LaplacianNoise, CustomNoise

from gymnasium import spaces
import numpy as np



### Gaussian

In [8]:

# Create a Gaussian noise model
env = DummyEnv()
env = SharedObsWrapper(env)
ns_model = GaussianNoise(mean=0.2, std=0.5)

env = NoiseWrapper(env, noise_model= ns_model)
# Reset and observe
env.reset()

obs0 = env.observe("agent0")
obs1 = env.observe("agent1")

print("Agent0 Observation:")
for k, v in obs0.items():
    print(f"{k}: {v}")

print("Agent1 Observation:")
for k, v in obs1.items():
    print(f"{k}: {v}")


Agent0 Observation:
agent0: [ 3. -1.]
agent1: [-0.30757792  4.68788465]
Agent1 Observation:
agent0: [ 3.03420878 -0.09989437]
agent1: [0. 4.]


#### Laplacian Model

In [3]:

# Create a Laplacian noise model
laplace_model = LaplacianNoise(loc=0.0, scale=0.2)

# Wrap with Laplacian noise
env = DummyNWEnv()
wrapped_env = NoiseWrapper(env, noise_model=laplace_model)

# Reset and observe
wrapped_env.reset()
agent = wrapped_env.agent_selection
obs = wrapped_env.observe(agent)

print(f"Laplacian noisy observation for {agent}:", obs)


NameError: name 'DummyNWEnv' is not defined

### Custom Noise Model
##### Allows definition of custom noise function

In [48]:
def custom_noise(cm_obs, space):
    return np.clip(cm_obs + 0.3, space.low, space.high)

cm_noise_model = CustomNoise(noise_fn=custom_noise)
env = DummyNWEnv()
wrapped_cm_env = NoiseWrapper(env, cm_noise_model)

wrapped_cm_env.reset()

agent = wrapped_cm_env.agent_selection
cm_obs = wrapped_cm_env.observe(agent)

print(f"Custom Noisy observation for {agent}:", cm_obs)



Custom Noisy observation for agent_0: [0.70000005 0.90000004]
